In [17]:
import pandas as pd
import os
!pip install pykrige

In [18]:
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging


In [19]:
nlags=20

In [20]:
import pandas as pd
from pykrige.ok import OrdinaryKriging
import numpy as np

def estimate_pollutant(df, date_column, nlags=20):
    # Create a DataFrame with only latitude, longitude, and the specified date column
    df1 = df[['longitude', 'latitude', date_column]]

    # Create a DataFrame with latitude and longitude where the specified date column's values are NaN
    nan_locations_df = df1[df1[date_column].isna()][['longitude', 'latitude']]

    # Create a DataFrame with latitude, longitude, and the specified date column excluding NaN values
    pollutant_df = df1.dropna(subset=[date_column])[['longitude', 'latitude', date_column]]

    # Prepare the data for Ordinary Kriging
    lons = np.array(pollutant_df['longitude'])
    lats = np.array(pollutant_df['latitude'])
    zdata = np.array(pollutant_df[date_column])

    # Perform Ordinary Kriging
    OK = OrdinaryKriging(lons, lats, zdata, variogram_model='linear', verbose=True, enable_plotting=False, nlags=nlags)
    grid_lon = np.array(nan_locations_df["longitude"])
    grid_lat = np.array(nan_locations_df["latitude"])
    z1, ss1 = OK.execute('grid', grid_lon, grid_lat)

    # Fill in NaN values in the specified date column with the interpolated values
    nan_locations_df[date_column] = z1[0]

    # Merge the two DataFrames based on their indices
    merged_df = pd.concat([pollutant_df, nan_locations_df], ignore_index=False)

    # Sort the DataFrame by ascending index
    merged_df = merged_df.sort_index(ascending=True)

    return merged_df

In [21]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_No2/Chosen_state_2023_sep_merge_sentinel5p_No2/Delhi_sep_No2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_No2/After_krigging_2023_sep_sentinal5p_No2/Delhi_2023_sep_No2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.275818420789994e-09
Nugget: 1.637352314258601e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.4868724467686448e-09
Nugget: 1.9961564059804036e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 3.0016892568533783e-10
Nugget: 2.52860888722137e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 4.3482481990170186e-11 

Calculating stati

In [22]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-27,2023-09-28,2023-09-29,2023-09-30
0,28.408461,76.832916,0.000013,0.000042,0.000013,7.851221e-07,1.598339e-05,0.000030,0.000015,5.162810e-06,...,0.000009,0.000040,0.000099,0.000013,0.000020,9.742981e-07,0.000011,0.000017,0.000005,8.061776e-06
1,28.422062,76.832916,0.000013,0.000030,0.000012,1.135144e-05,1.359849e-05,0.000030,0.000008,-2.578716e-06,...,-0.000031,0.000025,0.000009,0.000011,0.000025,3.139571e-05,0.000016,0.000002,0.000021,-1.252726e-06
2,28.435664,76.832916,0.000001,0.000036,-0.000044,9.215851e-07,1.187287e-05,0.000010,0.000033,3.072617e-06,...,0.000015,-0.000009,0.000016,0.000010,0.000043,1.960827e-05,0.000004,0.000012,0.000023,1.663727e-05
3,28.449265,76.832916,0.000008,0.000011,0.000012,9.215851e-07,1.214876e-06,0.000025,0.000018,-1.419294e-06,...,0.000015,0.000003,0.000014,0.000010,0.000029,2.302066e-05,0.000002,0.000008,0.000032,2.134110e-05
4,28.462866,76.832916,0.000013,0.000030,0.000015,-1.324731e-05,5.431245e-06,0.000012,0.000014,8.967905e-06,...,0.000011,-0.000005,-0.000021,0.000011,0.000027,2.190603e-05,0.000007,0.000009,0.000024,7.714142e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363,28.830106,77.328716,0.000025,0.000022,0.000008,-1.005415e-05,-9.851082e-07,0.000023,0.000026,5.685288e-06,...,0.000080,0.000017,0.000006,0.000012,0.000015,1.637433e-05,0.000009,0.000012,0.000041,1.117320e-05
1364,28.843708,77.328716,0.000016,0.000018,0.000020,9.390858e-06,-1.287443e-06,0.000022,0.000026,2.453048e-07,...,0.000011,0.000018,-0.000001,0.000032,0.000010,1.474426e-05,0.000010,0.000002,0.000021,4.456878e-05
1365,28.857309,77.328716,0.000019,0.000057,-0.000016,4.749173e-06,1.206551e-05,0.000012,0.000026,1.781609e-05,...,0.000025,0.000007,0.000069,0.000027,0.000020,3.130691e-06,0.000018,0.000010,0.000011,9.013133e-06
1366,28.870910,77.328716,0.000012,0.000057,0.000027,3.669032e-06,1.423883e-05,0.000007,0.000026,-2.199187e-06,...,0.000010,0.000010,0.000070,0.000006,0.000014,1.100059e-05,0.000009,-0.000001,0.000026,6.785802e-05


In [23]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_No2/Chosen_state_2023_sep_merge_sentinel5p_No2/WB_sep_No2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_No2/After_krigging_2023_sep_sentinal5p_No2/WB_2023_sep_No2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 2.2754921745991592e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.398743724517545e-10
Nugget: 2.299843625968142e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 3.563320502566243e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 4.191134108508609e-11 

Calculating statistics on variogram model fit...
Ex

In [24]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-27,2023-09-28,2023-09-29,2023-09-30
0,21.539446,85.826363,0.000025,0.000038,0.000031,0.000001,0.000025,0.000013,0.000009,-3.894893e-06,...,0.000027,-5.796801e-07,-8.314842e-07,0.000015,0.000024,0.000024,0.000009,-0.000004,0.000020,0.000012
1,21.660331,85.826363,0.000016,0.000042,0.000021,-0.000006,0.000010,0.000022,0.000023,-7.568676e-08,...,0.000009,4.803150e-05,1.894835e-06,0.000044,0.000023,0.000005,0.000015,0.000032,0.000020,0.000013
2,21.781215,85.826363,0.000016,0.000029,0.000023,0.000001,0.000009,0.000045,0.000030,2.664326e-06,...,0.000018,-3.133270e-06,1.281026e-06,0.000010,0.000089,0.000013,-0.000003,-0.000006,0.000004,0.000006
3,21.902100,85.826363,0.000035,0.000016,0.000034,0.000001,0.000010,0.000027,0.000063,2.843120e-06,...,-0.000014,1.274635e-05,6.444148e-06,0.000013,0.000012,0.000025,0.000024,0.000009,0.000038,0.000034
4,22.022985,85.826363,0.000028,0.000003,0.000023,-0.000004,0.000048,0.000008,0.000030,9.569584e-07,...,0.000011,2.595606e-05,1.019999e-05,0.000029,0.000070,0.000020,0.000023,0.000007,0.000019,0.000018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,26.737492,89.786363,0.000050,0.000006,0.000009,-0.000011,0.000016,0.000021,-0.000008,2.871382e-05,...,0.000024,1.740730e-05,6.000899e-06,0.000009,0.000002,0.000026,0.000006,0.000007,0.000020,0.000010
1628,26.858377,89.786363,0.000034,0.000006,0.000009,-0.000006,-0.000003,0.000026,0.000018,1.287261e-05,...,0.000012,2.849992e-05,5.405775e-06,0.000009,0.000022,0.000020,0.000007,0.000018,0.000037,0.000029
1629,26.979262,89.786363,0.000013,0.000026,0.000017,-0.000002,0.000033,0.000026,0.000003,1.419018e-05,...,0.000010,6.296583e-06,-2.365874e-06,0.000010,0.000002,0.000029,0.000013,0.000009,0.000025,0.000021
1630,27.100146,89.786363,0.000050,0.000016,0.000009,0.000020,0.000020,0.000023,0.000032,4.840569e-06,...,0.000014,2.008240e-05,1.566046e-05,0.000006,0.000014,0.000021,0.000017,0.000029,0.000016,0.000020


In [25]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_No2/Chosen_state_2023_sep_merge_sentinel5p_No2/karnataka_sep_No2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_No2/After_krigging_2023_sep_sentinal5p_No2/Karnataka_2023_sep_No2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 1e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 1.8350355689007968e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.3075763399440545e-10
Nugget: 1.2724876902761393e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 2.2459298544048833e-11 

Calculating statistics on variogram model fit...
Executing Ordin

In [26]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-27,2023-09-28,2023-09-29,2023-09-30
0,11.574480,74.054306,0.000012,0.000011,0.000024,0.000006,2.267027e-06,0.000034,0.000010,0.000006,...,6.530644e-06,6.636121e-06,-0.000006,7.756475e-06,0.000014,1.667625e-05,-0.000002,-8.464654e-07,2.758478e-05,0.000020
1,11.714901,74.054306,0.000012,0.000009,0.000009,0.000010,8.894076e-06,-0.000002,-0.000012,-0.000001,...,1.397371e-05,1.785967e-05,0.000004,1.647706e-05,0.000009,1.254908e-05,0.000004,6.742497e-06,4.419413e-05,0.000024
2,11.855323,74.054306,0.000012,0.000018,0.000041,-0.000006,8.526424e-06,0.000022,0.000002,-0.000004,...,1.827239e-05,9.557136e-07,0.000015,3.540752e-07,0.000020,8.561604e-06,0.000010,4.703154e-06,1.283822e-05,0.000027
3,11.995744,74.054306,0.000002,0.000011,0.000024,0.000002,-5.087257e-06,0.000018,0.000019,0.000002,...,5.800705e-07,1.082250e-06,0.000003,1.073900e-05,-0.000002,1.202443e-05,0.000015,-5.860704e-06,5.501217e-06,0.000014
4,12.136165,74.054306,0.000004,0.000007,0.000024,-0.000012,4.072991e-06,0.000005,-0.000011,0.000007,...,1.104900e-05,5.016641e-06,-0.000008,2.384833e-06,0.000011,5.814975e-06,0.000008,1.583956e-05,3.010925e-05,0.000028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1645,17.893435,78.534306,0.000026,0.000008,0.000009,0.000002,1.919875e-05,0.000020,0.000020,0.000011,...,1.253222e-05,7.740547e-06,-0.000006,3.032113e-05,0.000010,2.200709e-05,0.000009,-3.495059e-08,7.650494e-07,0.000011
1646,18.033856,78.534306,0.000019,0.000032,0.000026,0.000019,1.131414e-06,0.000013,0.000025,0.000005,...,2.449177e-05,4.964715e-06,0.000006,1.799283e-05,0.000029,1.032282e-05,0.000010,2.277078e-06,8.138777e-06,0.000015
1647,18.174278,78.534306,0.000013,0.000089,0.000020,0.000011,2.970559e-07,0.000031,0.000018,0.000004,...,2.569296e-05,3.491830e-06,0.000007,1.974484e-07,0.000017,8.705461e-07,0.000014,8.126046e-06,4.089115e-06,0.000020
1648,18.314699,78.534306,0.000011,0.000021,0.000026,0.000004,1.717001e-05,0.000036,0.000009,0.000011,...,6.980406e-06,1.539497e-05,0.000012,9.940867e-06,0.000011,5.009822e-05,0.000029,1.889884e-05,6.186346e-06,0.000019


In [27]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_No2/Chosen_state_2023_sep_merge_sentinel5p_No2/Orissa_sep_No2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_No2/After_krigging_2023_sep_sentinal5p_No2/Orissa_2023_sep_No2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 1e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 3.055142969776703e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.2328105412174673e-10
Nugget: 2.1915517405213574e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 1e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Ad

In [28]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-27,2023-09-28,2023-09-29,2023-09-30
0,17.802610,81.383003,0.000026,0.000014,0.000074,5.235889e-06,0.000036,0.000038,0.000020,7.381900e-06,...,0.000008,-0.000002,1.549839e-05,0.000029,-0.000017,0.000016,0.000008,8.633885e-06,0.000022,1.614992e-05
1,17.942639,81.383003,0.000028,0.000035,0.000032,5.544436e-06,0.000015,0.000010,0.000014,6.473378e-06,...,0.000004,0.000024,-1.546449e-07,0.000024,-0.000008,0.000016,0.000055,1.819421e-05,0.000017,9.966265e-06
2,18.082668,81.383003,0.000015,0.000027,0.000027,2.188778e-06,0.000015,0.000022,0.000020,1.126936e-05,...,0.000002,0.000031,4.871566e-06,0.000025,0.000015,0.000002,0.000025,9.372817e-06,0.000012,1.492940e-05
3,18.222697,81.383003,0.000012,0.000023,0.000013,4.346757e-09,0.000045,0.000020,-0.000017,1.605065e-05,...,0.000012,0.000029,2.695053e-06,0.000023,0.000033,0.000007,0.000005,4.779184e-06,0.000016,3.036806e-05
4,18.362726,81.383003,0.000013,0.000020,0.000036,6.142330e-06,0.000004,0.000007,0.000009,-7.310430e-07,...,0.000047,0.000048,1.754302e-05,0.000019,0.000015,0.000014,0.000015,4.138302e-05,0.000036,3.292966e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,22.003474,87.403003,0.000054,0.000043,0.000019,1.024932e-05,0.000008,0.000014,0.000007,-2.103421e-06,...,0.000032,-0.000010,2.823600e-05,0.000022,0.000021,0.000029,0.000021,7.346839e-07,0.000014,1.851536e-05
1536,22.143503,87.403003,0.000021,0.000061,0.000017,5.808173e-06,0.000002,0.000013,0.000041,1.128432e-05,...,0.000071,0.000023,1.283975e-05,0.000017,0.000007,0.000009,-0.000003,2.701736e-05,0.000018,2.469797e-05
1537,22.283532,87.403003,0.000028,0.000027,0.000013,1.572909e-07,0.000006,0.000013,0.000022,1.301793e-05,...,0.000032,0.000019,-4.052429e-06,0.000019,0.000015,0.000015,0.000016,1.092570e-05,0.000011,3.373522e-05
1538,22.423560,87.403003,0.000008,0.000022,0.000020,5.224002e-07,0.000055,0.000023,0.000014,-2.980682e-06,...,0.000022,0.000002,9.613270e-06,0.000025,0.000059,0.000010,0.000008,1.041254e-06,0.000015,1.380500e-05


In [29]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_No2/Chosen_state_2023_sep_merge_sentinel5p_No2/Tamilnadu_sep_No2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_No2/After_krigging_2023_sep_sentinal5p_No2/Tamilnadu_2023_sep_No2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 1.4570588111755152e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 9.915832941248876e-11 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.4746689826962088e-10
Nugget: 2.1684145181300665e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 2.542121580138941e-11 

Calculating statistics on variogram model fit...


In [30]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-27,2023-09-28,2023-09-29,2023-09-30
0,8.076526,76.225746,0.000006,-2.385165e-06,0.000012,2.191335e-06,8.724702e-06,-0.000003,-5.458418e-07,0.000010,...,3.372662e-07,0.000016,1.541741e-05,0.000006,0.000015,0.000008,0.000014,1.301905e-05,2.027618e-05,-0.000037
1,8.216553,76.225746,0.000013,3.470440e-06,0.000040,2.299807e-05,1.215780e-05,0.000015,7.686832e-06,0.000027,...,5.053932e-06,0.000010,-6.435403e-07,0.000003,0.000002,0.000024,-0.000002,-1.156215e-06,3.501181e-05,0.000027
2,8.356580,76.225746,0.000016,1.616644e-05,0.000029,5.285428e-06,-9.142572e-06,0.000004,-6.679904e-07,-0.000004,...,1.201272e-05,0.000017,6.558327e-06,-0.000002,0.000002,0.000012,0.000010,5.270664e-06,1.658758e-05,0.000012
3,8.496608,76.225746,0.000007,1.423382e-05,0.000004,1.570048e-06,3.888553e-07,0.000006,1.491964e-06,-0.000004,...,1.134679e-05,0.000009,-3.998304e-06,0.000005,0.000011,0.000018,0.000028,1.626161e-05,1.359012e-05,0.000014
4,8.636635,76.225746,0.000014,4.255507e-06,0.000014,3.076311e-06,9.437066e-06,0.000009,2.037261e-05,-0.000005,...,-6.378591e-06,0.000007,5.226004e-06,0.000004,-0.000005,0.000040,0.000018,6.090404e-06,2.814089e-05,0.000036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,12.977482,80.285746,0.000010,2.809721e-05,0.000021,3.075544e-07,-1.086015e-06,0.000018,-3.754361e-06,0.000012,...,1.359962e-05,-0.000001,9.861713e-06,0.000005,0.000024,0.000025,0.000012,-2.678251e-07,9.096229e-06,0.000012
1196,13.117509,80.285746,0.000022,4.074641e-06,0.000012,7.355691e-06,1.478983e-05,0.000021,1.299222e-05,0.000010,...,1.662864e-05,0.000006,4.355314e-06,0.000013,0.000031,0.000015,0.000014,1.623576e-05,7.973238e-06,0.001259
1197,13.257536,80.285746,0.000047,7.908515e-07,0.000008,-4.054620e-07,-2.160838e-05,0.000058,9.322414e-06,-0.000004,...,2.630393e-05,0.000010,7.373464e-06,0.000011,0.000013,0.000015,0.000024,9.515760e-06,2.157115e-05,0.000042
1198,13.397564,80.285746,0.000012,2.031194e-05,0.000030,3.622197e-06,4.334591e-06,0.000022,8.176906e-06,-0.000002,...,1.223338e-05,0.000010,-3.228222e-06,0.000014,0.000011,0.000409,0.000004,1.345999e-05,2.346873e-05,0.000019


In [31]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_No2/Chosen_state_2023_sep_merge_sentinel5p_No2/Gujrat_sep_No2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_No2/After_krigging_2023_sep_sentinal5p_No2/Gujrat_2023_sep_No2_final.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 1.7054993584211478e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.8527233623212896e-10
Nugget: 2.9372985094380055e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 3.718442348256607e-10
Nugget: 1e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 4.794202703082689e-11 

Calculating statistics on variogram model fit...


In [32]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-27,2023-09-28,2023-09-29,2023-09-30
0,20.120798,68.186249,0.000006,0.000013,0.000032,0.000007,4.201093e-06,0.000011,0.000006,1.910228e-05,...,0.000016,0.000007,3.277469e-06,0.000010,5.896991e-06,0.000011,0.000016,0.000010,0.000003,0.000019
1,20.264079,68.186249,-0.000010,0.000005,0.000023,0.000013,-1.186602e-07,0.000020,0.000006,1.630118e-05,...,0.000003,0.000007,-4.315019e-06,0.000008,-5.562078e-07,0.000010,0.000020,0.000014,0.000026,0.000012
2,20.407360,68.186249,0.000012,0.000035,0.000022,0.000003,1.759527e-07,0.000005,0.000016,2.708219e-05,...,0.000012,0.000005,-6.157009e-07,0.000023,1.001894e-05,0.000034,0.000009,0.000011,0.000015,-0.000009
3,20.550641,68.186249,0.000012,0.000010,0.000010,0.000012,-2.954875e-07,0.000023,0.000010,9.870027e-06,...,0.000007,-0.000011,1.394554e-05,0.000007,-1.340889e-05,0.000013,0.000001,0.000007,0.000014,0.000024
4,20.693922,68.186249,0.000018,0.000022,0.000025,0.000003,2.616280e-06,0.000017,0.000016,1.035477e-05,...,0.000018,-0.000002,6.823776e-06,0.000014,1.393175e-05,0.000037,0.000025,0.000010,0.000010,0.000024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,24.132666,74.346249,0.000023,0.000024,0.000028,0.000006,-5.706348e-07,0.000042,0.000024,2.535016e-05,...,0.000026,0.000010,2.647922e-05,0.000017,1.857107e-05,0.000016,0.000004,0.000012,0.000019,0.000025
1481,24.275947,74.346249,0.000023,0.000024,0.000021,0.000004,-1.490600e-05,0.000020,0.000001,8.280217e-07,...,0.000058,-0.000019,4.266327e-06,0.000009,1.113791e-05,0.000016,0.000028,0.000010,0.000043,0.000006
1482,24.419228,74.346249,0.000019,0.000022,0.000023,0.000004,9.767416e-06,0.000034,0.000019,7.128289e-06,...,0.000007,0.000015,1.273460e-06,0.000003,5.120689e-06,0.000026,0.000003,0.000002,0.000025,0.000011
1483,24.562509,74.346249,0.000033,0.000040,0.000030,0.000011,-2.136235e-07,0.000008,0.000015,4.504200e-06,...,-0.000001,-0.000024,2.368862e-05,0.000021,-5.500239e-07,0.000012,0.000012,0.000021,0.000014,0.000008


In [33]:
# df should be the DataFrame with columns containing NaN values
df = pd.read_csv("/home/sysadm/Downloads/sentinal5p_No2/Chosen_state_2023_sep_merge_sentinel5p_No2/Maharashtra_sep_No2.csv")

# Get the list of columns that have NaN values
columns_with_nan = df.columns[df.isna().any()].tolist()

# Create an empty dictionary to store the estimated AOD DataFrames
estimated_data = {}

# Iterate through the columns with NaN values and apply estimate_aod
for date_column in columns_with_nan:
    estimated_data[date_column] = estimate_pollutant(df, date_column)

# Create a list of columns to keep in the estimated_df
columns_to_keep = ['longitude', 'latitude'] + columns_with_nan

# Select only the desired columns from the estimated_df
estimated_df = pd.concat([estimated_data[date_column] for date_column in columns_with_nan], axis=1)
estimated_df = estimated_df[columns_to_keep]

# Drop the duplicate 'latitude' and 'longitude' columns
estimated_df = estimated_df.loc[:,~estimated_df.columns.duplicated()]

# Create a DataFrame 'original_df' containing the original columns without NaN values
original_columns = df.columns.difference(columns_with_nan).tolist()
original_df = df[original_columns]
# Merge the estimated_df and original_df based on 'latitude' and 'longitude'
result_df = original_df.merge(estimated_df, on=['longitude', 'latitude'], how = 'inner')
result_df.to_csv("/home/sysadm/Downloads/sentinal5p_No2/After_krigging_2023_sep_sentinal5p_No2/Maharashtra_sep_No2.csv",index = False)

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 1e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 1e-10 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1.2616214313302011e-10
Nugget: 2.1595097001031552e-09 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 1e-10
Nugget: 3.7607996367649406e-11 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...

A

In [34]:
result_df

,latitude,longitude,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06,2023-09-07,2023-09-08,...,2023-09-20,2023-09-21,2023-09-22,2023-09-23,2023-09-24,2023-09-25,2023-09-27,2023-09-28,2023-09-29,2023-09-30
0,15.604600,72.650696,0.000001,0.000013,0.000023,6.804969e-06,-0.000004,0.000017,7.667274e-06,5.069568e-06,...,-2.332283e-06,0.000005,0.000003,-1.378985e-06,0.000008,0.000023,0.000011,0.000007,0.000017,0.000035
1,15.799339,72.650696,0.000010,0.000007,0.000021,7.788952e-06,0.000003,0.000013,7.667274e-06,-6.098878e-06,...,1.711724e-06,-0.000002,0.000002,-5.965477e-07,0.000013,0.000016,0.000020,-0.000009,0.000016,0.000229
2,15.994079,72.650696,0.000013,0.000002,0.000071,-3.177468e-07,0.000002,0.000033,6.624751e-06,1.758159e-05,...,9.578583e-06,0.000003,-0.000002,7.610586e-06,0.000008,0.000014,0.000040,0.000003,0.000014,0.000053
3,16.188818,72.650696,0.000006,0.000024,0.000016,6.924102e-06,-0.000014,0.000011,8.240914e-06,6.037349e-06,...,-1.607780e-05,0.000008,0.000010,4.057011e-06,-0.000015,0.000014,0.000025,0.000010,0.000025,-0.000012
4,16.383557,72.650696,0.000010,-0.000005,0.000033,1.914016e-06,0.000006,0.000061,2.658427e-05,2.011336e-05,...,8.780992e-07,0.000006,0.000010,1.100637e-05,0.000033,0.000094,0.000011,0.000003,0.000015,0.000014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,21.252041,80.820696,0.000023,0.000025,0.000029,8.632253e-06,-0.000013,0.000021,2.853772e-05,1.611068e-05,...,2.468306e-05,0.000023,0.000014,1.083759e-05,0.000028,0.000016,0.000008,0.000021,0.000034,0.000007
1492,21.446780,80.820696,0.000020,0.000015,0.000023,1.336651e-05,-0.000013,0.000013,1.102747e-05,9.546242e-07,...,-2.221506e-06,-0.000002,0.000016,2.932904e-05,0.000014,0.000033,0.000020,0.000015,0.000010,0.000004
1493,21.641520,80.820696,0.000023,0.000019,0.000009,8.632253e-06,0.000007,0.000037,1.756347e-07,1.022456e-06,...,1.518520e-05,0.000019,0.000026,1.756258e-05,0.000033,0.000009,0.000023,0.000005,0.000010,0.000013
1494,21.836259,80.820696,0.000039,0.000027,0.000029,8.632253e-06,0.000003,0.000018,2.353436e-05,-7.223454e-06,...,4.025842e-06,0.000010,0.000009,1.297636e-05,0.000014,0.000019,0.000012,0.000038,0.000047,0.000018
